In [1]:
# Import packages and libraries
import numpy as np
import random as rnd

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from pprint import pprint

from Semi_EM_NB import Semi_EM_MultinomialNB

In [2]:
# Load train and test data set with class labels 
train_Xy = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
test_Xy = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [3]:
# Convert all text data into tf-idf vectors
vectorizer = TfidfVectorizer()
train_vec = vectorizer.fit_transform(train_Xy.data)
test_vec = vectorizer.transform(test_Xy.data)

In [4]:
# Divide train data set into labeled and unlabeled data sets
n_train_data = train_vec.shape[0]
split_ratio = 0.3 # labeled vs unlabeled
X_l, X_u, y_l, y_u = train_test_split(train_vec, train_Xy.target, train_size=split_ratio)
print X_l.shape, X_u.shape

(3394, 101631) (7920, 101631)


In [5]:
# Train Naive Bayes classifier (imported) 
# using both labeled and unlabeled data set
clf = Semi_EM_MultinomialNB(alpha=1e-8) # semi supervised EM based Naive Bayes classifier
clf.fit(X_l, y_l, X_u)

-8839550.93803
1
-6470076.98986
2
-6469392.1663
3
-6469387.6444
4
-6469392.18267


<Semi_EM_NB.Semi_EM_MultinomialNB instance at 0x10a6a3e18>

In [6]:
# Evaluate NB classifier using test data set
pred = clf.predict(test_vec)
print(metrics.classification_report(test_Xy.target, pred, target_names=test_Xy.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.58      0.14      0.22       319
           comp.graphics       0.58      0.41      0.48       389
 comp.os.ms-windows.misc       0.54      0.18      0.26       394
comp.sys.ibm.pc.hardware       0.56      0.56      0.56       392
   comp.sys.mac.hardware       0.59      0.51      0.55       385
          comp.windows.x       0.52      0.77      0.62       395
            misc.forsale       0.80      0.39      0.53       390
               rec.autos       0.73      0.62      0.67       396
         rec.motorcycles       0.80      0.42      0.55       398
      rec.sport.baseball       0.93      0.65      0.76       397
        rec.sport.hockey       0.84      0.84      0.84       399
               sci.crypt       0.51      0.71      0.59       396
         sci.electronics       0.73      0.32      0.44       393
                 sci.med       0.67      0.70      0.68       396
         

In [7]:
pprint(metrics.confusion_matrix(test_Xy.target, pred))

array([[ 44,   1,   0,   0,   1,   2,   0,   0,   4,   2,   4,   8,   1,
          6,  22, 167,   7,  25,  21,   4],
       [  1, 158,  11,   9,  11,  92,   2,   3,   2,   1,   0,  20,   0,
          6,  62,   8,   1,   0,   1,   1],
       [  1,  35,  69,  65,  22,  97,   1,   2,   0,   0,   1,  31,   3,
         10,  42,   8,   1,   1,   5,   0],
       [  0,  14,  15, 221,  52,  20,  10,   2,   1,   0,   1,  16,  11,
          1,  26,   0,   1,   0,   1,   0],
       [  0,  13,   9,  34, 197,  22,   8,   4,   1,   0,   0,  12,   9,
          9,  60,   4,   1,   1,   1,   0],
       [  0,  23,   3,   7,   4, 305,   2,   0,   0,   0,   0,  15,   0,
         10,  24,   0,   1,   1,   0,   0],
       [  0,   7,   5,  36,  32,  13, 154,  17,   6,   2,   8,   3,   6,
          5,  77,   4,   4,   6,   5,   0],
       [  0,   1,   1,   0,   1,   2,   5, 246,  16,   1,   1,  12,   7,
         10,  58,   9,   8,   5,  13,   0],
       [  2,   3,   2,   0,   2,   1,   2,  43, 168,   1,   4,  

In [8]:
print(metrics.accuracy_score(test_Xy.target, pred))

0.54487519915


In [9]:
b_w_d = X_u>0
type(b_w_d)

scipy.sparse.csr.csr_matrix

In [10]:
b_w_d.dot(b_w_d.transpose())

<7920x7920 sparse matrix of type '<type 'numpy.bool_'>'
	with 57145955 stored elements in Compressed Sparse Row format>

In [11]:
np.concatenate((y_l, y_l), axis=0).shape

(6788,)